In [3]:
import numpy
import pandas
import matplotlib.pyplot as plt
import ESN_Modules as ESN
from scipy import linalg
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import PowerTransformer

In [10]:
pt = PowerTransformer()
data = ESN.reshape(numpy.loadtxt("WEGEData.txt", delimiter = ","))
pt.fit(data)
ndata = pt.transform(data)
numpy.savetxt("WEGETrainData.txt", ndata, delimiter = ",")
#print(pt.inverse_transform(ndata))
ndata

array([[-0.11490094,  0.5815316 , -1.06490635,  0.09604408],
       [-0.85043792, -1.97935992,  0.7031509 ,  0.69715065],
       [-1.85021713, -0.6003687 , -1.06573356, -1.17719202],
       ...,
       [-0.35162729, -0.47175386, -0.58479169, -0.59191457],
       [-0.23211023, -0.81517235, -0.01170137, -0.59264066],
       [ 1.1855641 ,  1.51482814,  1.03594002,  1.52257019]])

In [11]:
def train_parameters(X, train_length, transiente, N, K, scale, radius, sparsity, leak):
    numpy.random.seed(X)
    # PESOS DE ENTRADA
    W_input = ESN.input_weights(N = N, K = K, input_scaling = scale)
    # PESOS DE RESERVATORIO
    W_reservoir = ESN.reservoir_weights(N = N, spectral_radius = radius, sparsity = sparsity)

    # MATRIZ DE ESTADOS (TRAINNING)
    state_matrix = ESN.harvesting_states(state_matrix = ESN.create_states(dim = 1 + K + N, patterns = train_length - transiente), 
                                     input_patterns = train_dataset, 
                                     win = W_input, 
                                     wres = W_reservoir, 
                                     initial_state = numpy.zeros((N, 1)),
                                     leaking = leak, 
                                     transiente = transiente)

    # PESOS DE SAÍDA
    W_output = ESN.ridge_regression(states = state_matrix, target = train_target)
    # ERRO DE TREINO
    return(numpy.dot(W_output, state_matrix))

In [12]:
# TAMANHO DE JANELA DE ENTRADA (INPUT)
len_window_input = 20
# TAMANHO DE JANELA DE SAÍDA (OUTPUT)
len_window_output = 1

# PASSOS AHEAD QUE DESEJAMOS PREVER
steps = 1

# NÚMERO DE NEURÔNIOS NA CAMADA OCULTA, INT, N > 0
neurons = [20]

# NÚMERO DE HIPERPARÂMETROS A SEREM TESTADOS POR CATEGORIA
spam = 10
# NÚMERO DE RODADAS
rounds = 10

# TAXA DE VAZAMENTO ESPECTRAL, FLOAT, NO CONJUNTO (0, 1]
leaking_rates = numpy.linspace(start = 0.05, stop = 0.95, num = spam).round(2)
# ESPARSIDADE DA MATRIZ, FLOAT, NO CONJUNTO [0, 1]
sparsities = numpy.linspace(start = 0.05, stop = 0.95, num = spam).round(2)
# ESCALA DOS DADOS DE ENTRADA, FLOAT, NO CONJUNTO (0, INF)
input_scalings = numpy.linspace(start = 0.1, stop = 1.9, num = spam).round(2)
# RAIO ESPECTRAL DESEJADO, FLOAT, NO CONJUNTO (0, INF)
spectral_radius = numpy.linspace(start = 0.1, stop = 1.9, num = spam).round(2)

In [13]:
file_name = "WEGETrainData.txt"

# OUTPUT ESPERADO
output_dataset, L = ESN.set_window2(name = file_name, window_len = len_window_output, begin = len_window_input)

# INPUT DO SISTEMA
input_dataset, K = ESN.set_window2(name = file_name, window_len = len_window_input, begin = 0, steps = steps * len_window_output)

dim = input_dataset.shape[0]

# TAMANHO DA JANELA DE TRAINNING
train_length = int(0.7*dim)
# TAMANHO DA JANELA DE TESTING
test_length = dim - train_length - steps

# TRANSIENTE, VALORES A SEREM DESCARTADOS
transiente = int(train_length*0.1)

# DATASET DE TREINO -> [(X0), (X1), (X2), ..., (Xjanela_treino-1)]
train_dataset = input_dataset[0 : train_length, :]

# DATASET DE TREINO (TARGET) -> [(Ysteps+transiente), (Ysteps+transiente+1), ..., (Ysteps+janela_treino-1)]
train_target = output_dataset[None, steps + transiente - 1 : train_length + steps - 1, :]

# DATASET DE TESTE -> [(Xjanela_treino), (Xjanela_treino+1), (Xjanela_treino+2), ..., (Xjanela_treino+janela_teste-1)]
test_dataset = input_dataset[train_length : train_length + test_length, :]

# DATASET DE TESTE (TARGET) -> [(Ysteps+janela_treino+janela_teste), ..., (Ysteps+janela_treino+janela_teste-1)]
test_target = output_dataset[train_length + steps - 1: train_length + test_length, :]

eval_model = train_target[0].T

In [14]:
# DEFINE UMA SEMENTE DO GERADOR PSEUDO-ALEATORIO
#numpy.random.seed(69)
numpy.random.seed(87)

cols = ["ROUND", "WINDOW", "NEURONS", "LEAK", "SPARSITY", "SCALE", "RADIUS", "MSE", "RMSE", "MAPE", 
        "MSE(X)", "RMSE(X)", "MAPE(X)", "MSE(Y)", "RMSE(Y)", "MAPE(Y)", "MSE(Z)", "RMSE(Z)", "MAPE(Z)",
        "FRAMEWORK"]

sample = (spam**4) * len(neurons) * rounds
architectures = numpy.random.randint(low = 1, high = sample * 2, size = sample)
simulation = pandas.DataFrame(numpy.nan, columns = cols, 
                              index = numpy.linspace(start = 0, stop = sample - 1, num = sample).astype(int))
i = 0

for rnd in range(rounds):
    print(rnd)
    for N in neurons:
        for leak in leaking_rates:
            for sparsity in sparsities:
                for scale in input_scalings:
                    for radius in spectral_radius:
                        predict = train_parameters(
                            X = architectures[i], 
                            train_length = train_length, 
                            transiente = transiente, 
                            N = N, 
                            K = K, 
                            scale = scale, 
                            radius = radius, 
                            sparsity = sparsity, 
                            leak = leak
                        )
                        simulation.iloc[i,:] = [rnd, len_window_input, N, leak, sparsity, scale, radius] + ESN.model_metrics(eval_model, predict) + [architectures[i]]
                        i += 1

0
1
2
3
4
5
6
7
8
9


In [15]:
simulation

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
0,0.0,20.0,20.0,0.05,0.05,0.1,0.1,0.947005,0.681403,2.002526,0.954186,0.976824,2.600297,0.845462,0.919490,2.268162,1.022714,1.011293,1.656070,60903.0
1,0.0,20.0,20.0,0.05,0.05,0.1,0.3,0.947875,0.681309,2.006709,0.955620,0.977558,2.539816,0.846538,0.920075,2.283574,1.023616,1.011739,1.673980,41294.0
2,0.0,20.0,20.0,0.05,0.05,0.1,0.5,0.946900,0.681026,2.031955,0.953779,0.976616,2.643594,0.847353,0.920518,2.317792,1.019391,1.009649,1.678220,27324.0
3,0.0,20.0,20.0,0.05,0.05,0.1,0.7,0.946975,0.681238,2.032452,0.952989,0.976211,2.633321,0.849595,0.921735,2.282840,1.019225,1.009567,1.701497,136199.0
4,0.0,20.0,20.0,0.05,0.05,0.1,0.9,0.945737,0.681333,2.021108,0.953512,0.976480,2.586758,0.845939,0.919750,2.256594,1.019268,1.009588,1.752967,95585.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,9.0,20.0,20.0,0.95,0.95,1.9,1.1,0.951078,0.682619,1.986508,0.958622,0.979092,2.533628,0.852119,0.923103,2.255606,1.023692,1.011777,1.683559,59882.0
99996,9.0,20.0,20.0,0.95,0.95,1.9,1.3,0.948352,0.682778,2.030681,0.955181,0.977334,2.552404,0.850903,0.922444,2.261725,1.019814,1.009859,1.799904,64243.0
99997,9.0,20.0,20.0,0.95,0.95,1.9,1.5,0.950955,0.683738,2.014770,0.958356,0.978957,2.581188,0.851424,0.922726,2.277635,1.025038,1.012442,1.693577,89641.0
99998,9.0,20.0,20.0,0.95,0.95,1.9,1.7,0.952373,0.683330,1.978815,0.959452,0.979516,2.508342,0.853881,0.924057,2.269799,1.024511,1.012181,1.675625,80233.0


In [16]:
simulation[(simulation.LEAK == 0.05) & (simulation.SCALE == 0.1) & (simulation.SPARSITY == 0.55) & (simulation.RADIUS == 0.5)]

,ROUND,WINDOW,NEURONS,LEAK,SPARSITY,SCALE,RADIUS,MSE,RMSE,MAPE,MSE(X),RMSE(X),MAPE(X),MSE(Y),RMSE(Y),MAPE(Y),MSE(Z),RMSE(Z),MAPE(Z),FRAMEWORK
502,0.0,20.0,20.0,0.05,0.55,0.1,0.5,0.949618,0.682121,2.023625,0.956279,0.977895,2.592477,0.850262,0.922096,2.301078,1.021137,1.010513,1.733820,170336.0
10502,1.0,20.0,20.0,0.05,0.55,0.1,0.5,0.947874,0.681524,1.988435,0.952433,0.975927,2.520368,0.847752,0.920735,2.290077,1.022361,1.011119,1.674492,136100.0
20502,2.0,20.0,20.0,0.05,0.55,0.1,0.5,0.951432,0.682345,1.982992,0.959071,0.979322,2.538313,0.852017,0.923048,2.258517,1.024821,1.012335,1.666113,120052.0
30502,3.0,20.0,20.0,0.05,0.55,0.1,0.5,0.947788,0.681859,2.032123,0.956915,0.978221,2.617679,0.847136,0.920400,2.317139,1.021875,1.010878,1.675383,97412.0
40502,4.0,20.0,20.0,0.05,0.55,0.1,0.5,0.944402,0.682108,2.091407,0.949585,0.974467,2.795584,0.844480,0.918956,2.330549,1.020727,1.010311,1.701456,15896.0
50502,5.0,20.0,20.0,0.05,0.55,0.1,0.5,0.948829,0.681289,1.992281,0.955641,0.977569,2.532771,0.851264,0.922640,2.261857,1.021002,1.010446,1.686065,116028.0
60502,6.0,20.0,20.0,0.05,0.55,0.1,0.5,0.947683,0.681242,2.039562,0.953466,0.976456,2.645626,0.849579,0.921726,2.284642,1.019383,1.009645,1.744555,106541.0
70502,7.0,20.0,20.0,0.05,0.55,0.1,0.5,0.945409,0.681233,2.050605,0.951168,0.975278,2.636691,0.844762,0.919109,2.316160,1.021036,1.010463,1.720804,3940.0
80502,8.0,20.0,20.0,0.05,0.55,0.1,0.5,0.946201,0.680713,2.063863,0.950611,0.974993,2.713959,0.845137,0.919314,2.303638,1.020365,1.010131,1.756157,138691.0
90502,9.0,20.0,20.0,0.05,0.55,0.1,0.5,0.945988,0.680411,2.044742,0.950271,0.974819,2.681098,0.848939,0.921379,2.268654,1.018270,1.009093,1.736534,101456.0


In [17]:
architectures

array([ 60903,  41294,  27324, ...,  89641,  80233, 111929])

In [18]:
simulation.to_csv("ParametrosTreinoReal_WEGE.txt", index = False)